# Twitter data collection

#### References

searchtweets API reference: https://twitterdev.github.io/search-tweets-python/  
Twitter API reference: https://developer.twitter.com/en/docs/tweets/search/api-reference/premium-search.html  
Twitter tweet object and dictionary: https://developer.twitter.com/en/docs/tweets/data-dictionary/overview/tweet-object

## Imports and credentials

In [4]:
from searchtweets import ResultStream, gen_rule_payload, load_credentials, collect_results

# general imports
import numpy as np
import pandas as pd
from textblob import TextBlob
import re
import time
import datetime

# plotting and visualization
import matplotlib
import matplotlib.pyplot as plt
%matplotlib inline

premium_search_args_30day = load_credentials("~/.twitter_keys.yaml",
                                          yaml_key="search_tweets_premium_30day",
                                          env_overwrite=False)
premium_search_args_fullarchive = load_credentials("~/.twitter_keys.yaml",
                                          yaml_key="search_tweets_premium_fullarchive",
                                          env_overwrite=False)

Grabbing bearer token from OAUTH
Grabbing bearer token from OAUTH


## Functions

In [5]:
def days_to_collect(start, end, frequency):
    '''
    will return an array starting at midnight of desired date to last frequency hour of end date
    start = start date
    end = end date
    frequency = number of hours to step by per day. For example frequency = 12, will collect twice: at midnight and noon
    '''
    # add one day for right_side border case
    # pd.date_range only allows dates, use rounding dates and closed='right' to get desired dates
    #print(start, end)
    start = datetime.datetime.strptime(start, '%Y-%m-%d') - datetime.timedelta(days=0, hours=int(frequency))
    end = datetime.datetime.strptime(end, '%Y-%m-%d') + datetime.timedelta(days=1, hours=0)
    #print(start, end)
    dates = pd.date_range(start=start, end=end, freq=frequency+'H', closed='left')
    formatted_dates = [ datetime.datetime.strftime(t, '%Y%m%d%H%M') for t in dates ]
    #print(formatted_dates)
    return formatted_dates

def collect_tweets(from_date, to_date, max_results):
    # maxResults is capped at 100 for sandbox account, even though there should be a next function to get more, it 
    # appears max_results=500 is accepted without any extra work
    # date format: YYYY-mm-DD HH:MM
    # from_date is inclusive. to_date is non-inclusive. Appears to start at from_date and start collecting tweets working
    # backwards to to_date
    bitcoin_rule = gen_rule_payload("bitcoin", results_per_call=100, from_date=from_date, to_date=to_date) 
    print(bitcoin_rule)
    collected_tweets = collect_results(bitcoin_rule, max_results=max_results, result_stream_args=premium_search_args)
    return collected_tweets

## Dates

In [13]:
example_start_date = '2018-10-11'
example_end_date = '2018-10-15'
interval = 12
results_per_call=100
max_results = 500

print("The intra-day hour interval is set to", interval, "Edit the code if desired to change this")
print("The number of tweets per interval is set to", max_results, "Edit the code if desired to change this")
print("please input two dates in the format below to collect dates\n\t", example_start_date, example_end_date, "\n")
user_dates = input("\t")
print()
start_date, end_date = user_dates.split(' ')
test_dates = days_to_collect(start_date, end_date, str(interval))

if (datetime.datetime.fromtimestamp(time.time()) - datetime.datetime.strptime(start_date, '%Y-%m-%d')).days < 30:
    premium_search_args = premium_search_args_30day
    print("will use 30-day dev environment")
else:
    premium_search_args = premium_search_args_fullarchive
    print("will use full-archive dev environment")
    
print("\ntwitter recognized dates will be collected on the closed iterval from", start_date, "to", end_date, "spaced in", str(interval), "hour intervals")

The intra-day hour interval is set to 12 Edit the code if desired to change this
The number of tweets per interval is set to 500 Edit the code if desired to change this
please input two dates in the format below to collect dates
	 2018-10-11 2018-10-15 

	2018-12-01 2018-12-15

will use 30-day dev environment

twitter recognized dates will be collected on the closed iterval from 2018-12-01 to 2018-12-15 spaced in 12 hour intervals


## Twitter call

In [14]:
tweets = []
for i in range(0,len(test_dates[:-1])):
    tweets = np.append(tweets, collect_tweets(test_dates[i], test_dates[i+1], max_results=max_results))
    
    # Requests are limited to 30 per minute for sandbox, 60 for subscriptions 
    # Requests are limited to 10 per second
    num_calls = (i + 1) * max_results//results_per_call
    if num_calls % 10 == 0 and num_calls % 20 != 0:
        print("waiting 2 seconds")
        time.sleep(2)
    if num_calls % 20 == 0:
        print("waiting 60 seconds")
        time.sleep(60)

{"query": "bitcoin", "maxResults": 100, "toDate": "201812010000", "fromDate": "201811301200"}
{"query": "bitcoin", "maxResults": 100, "toDate": "201812011200", "fromDate": "201812010000"}
waiting 2 seconds
{"query": "bitcoin", "maxResults": 100, "toDate": "201812020000", "fromDate": "201812011200"}
{"query": "bitcoin", "maxResults": 100, "toDate": "201812021200", "fromDate": "201812020000"}
waiting 60 seconds
{"query": "bitcoin", "maxResults": 100, "toDate": "201812030000", "fromDate": "201812021200"}
{"query": "bitcoin", "maxResults": 100, "toDate": "201812031200", "fromDate": "201812030000"}
waiting 2 seconds
{"query": "bitcoin", "maxResults": 100, "toDate": "201812040000", "fromDate": "201812031200"}
{"query": "bitcoin", "maxResults": 100, "toDate": "201812041200", "fromDate": "201812040000"}
waiting 60 seconds
{"query": "bitcoin", "maxResults": 100, "toDate": "201812050000", "fromDate": "201812041200"}
{"query": "bitcoin", "maxResults": 100, "toDate": "201812051200", "fromDate": "2

In [15]:
tweets[-1]

{'created_at': 'Sat Dec 15 11:45:10 +0000 2018',
 'id': 1073906811026063361,
 'id_str': '1073906811026063361',
 'text': 'RT @griffexco: You’d be making a mistake, if you are dismissing this fact, cause its against the popular belief. Its not just a coincidence…',
 'source': '<a href="http://twitter.com" rel="nofollow">Twitter Web Client</a>',
 'truncated': False,
 'in_reply_to_status_id': None,
 'in_reply_to_status_id_str': None,
 'in_reply_to_user_id': None,
 'in_reply_to_user_id_str': None,
 'in_reply_to_screen_name': None,
 'user': {'id': 1032064558381162497,
  'id_str': '1032064558381162497',
  'name': 'Andi Karl',
  'screen_name': 'andikarl77',
  'location': None,
  'url': None,
  'description': 'Airdrop & Bounty Hunter',
  'translator_type': 'none',
  'protected': False,
  'verified': False,
  'followers_count': 30,
  'friends_count': 81,
  'listed_count': 0,
  'favourites_count': 30,
  'statuses_count': 86,
  'created_at': 'Wed Aug 22 00:39:00 +0000 2018',
  'utc_offset': None,


## To dataframe and csv

In [16]:
def to_df(tweets):
    # create a pandas df from tweets
    S2 = pd.DataFrame(columns=['tweets', 'date', 'user_name', 'user_screen_name', 'user_followers', 
                           'user_friends', 'user_verified', 'user_language', 'retweet_count', 'favorite_count'])

    for i, tweet in enumerate(tweets):
        S2.loc[i] = [tweet['text'], 
                     tweet['created_at'], 
                     tweet['user']['name'], 
                     tweet['user']['screen_name'], 
                     tweet['user']['followers_count'], 
                     tweet['user']['friends_count'], 
                     tweet['user']['verified'], 
                     tweet['user']['lang'], 
                     tweet['retweet_count'], 
                     tweet['favorite_count']] 
    return S2

S2 = to_df(tweets)

In [17]:
S2.tail()

,tweets,date,user_name,user_screen_name,user_followers,user_friends,user_verified,user_language,retweet_count,favorite_count
14995,RT @Nouriel: Bitcoin &amp; crypto-currencies m...,Sat Dec 15 11:45:14 +0000 2018,NavigateMarketStorm,TheMarketStorm,5943,3869,False,en,0,0
14996,Earn #Bitcoin with the new CryptoTab Browser! ...,Sat Dec 15 11:45:14 +0000 2018,Bit • ΞTH Finder,bitethfinder,1588,1589,False,en,0,0
14997,🔄 Prices update in $USDT (1 hour):\n\n$BTC - 3...,Sat Dec 15 11:45:14 +0000 2018,BittrexPrices,BittrexPrices,752,656,False,en,0,0
14998,Trade OLXA on @tradekainos #Cryptocurrency #Ex...,Sat Dec 15 11:45:10 +0000 2018,OlxaCoin,OlxaCoin,6782,7279,False,en,0,3
14999,"RT @griffexco: You’d be making a mistake, if y...",Sat Dec 15 11:45:10 +0000 2018,Andi Karl,andikarl77,30,81,False,en,0,0


In [12]:
# save file to csv
S2_tweets = S2.loc[:,['tweets']]
S2_meta = S2.drop(['tweets'], axis=1)

filename = 'complete_tweets/tweets_' + start_date + '_' + end_date
S2_tweets.to_csv(filename + '_Tweets.csv', index=False)
S2_meta.to_csv(filename + '_Metadata.csv', index=False)
print('saved files', filename + '_Tweets.csv', 'and', filename + '_Metadata.csv')

saved files complete_tweets/tweets_2018-12-01_2018-12-01_Tweets.csv and complete_tweets/tweets_2018-12-01_2018-12-01_Metadata.csv
